In [ ]:
%idle_timeout 2880
%glue_version 4.0
%region us-east-1
%number_of_workers 2
%worker_type G.1X
%session_id_prefix fantasy-football

In [ ]:
%%configure
{
    'datalake-formats': 'delta'
}

In [ ]:
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from pyspark.sql.functions import col, lit, rank
from pyspark.sql.window import Window

# When running locally, if you enounter the following error:
# ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=GlueReplApp, master=jes) created by __init__ at /tmp/3143887023558129831:505 
# 
# Comment out the sc = SparkContext().getOrCreate() line and try again.
sc = SparkContext().getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

In [ ]:
BUCKET_NAME='madhav-delta-lake-project-1'

In [ ]:
df_total = glueContext.create_data_frame.from_catalog(
    database="delta_lake",
    table_name="top_performers_delta")

In [ ]:
import boto3

s3_resource = boto3.resource('s3')

In [ ]:
# Iterate through files in an s3 folder with boto3
bucket = s3_resource.Bucket(f'{BUCKET_NAME}')

for obj in bucket.objects.filter(Prefix='database/raw/', ):

    if not obj.key.endswith('.csv'):
        continue

    # Get year from file name
    year = int(obj.key.split('/')[2].split('.')[0])

    # Read csv file from s3 and convert to DataFrame
    df = glueContext.create_data_frame.from_options(
        connection_type='s3',
        connection_options={'paths': [f's3a://{BUCKET_NAME}/{obj.key}']},
        format='csv',
        format_options={'withHeader': True},
        transformation_ctx='datasource0')
    
    df = df.withColumn("FantasyPoints", df["FantasyPoints"].cast("double"))
    
    # Create a window specification to get top 12 players by position
    # Note: This will actually return all players that have a top 12 fantasy score
    # which means that there may be more than 12 players for a given position
    windowSpec = Window.partitionBy('Pos').orderBy(df['FantasyPoints'].desc())
    
    df_csv = df.filter(col("Pos").isin(["QB", "WR", "RB"])) \
        .select('Pos', 'Player', 'FantasyPoints') \
        .withColumn("rank", rank().over(windowSpec)) \
        .filter(col("rank") <= 12)
    
    # Clean up additional columns and add year
    df_csv = df_csv.withColumnRenamed("Pos", "Position") \
        .withColumn("Year", lit(year)) \
        .select("Year", "Position", "Player", "FantasyPoints")

    df_total = df_total.union(df_csv)

In [ ]:
# Write pyspark dataframe to delta lake
df_total.write.format("delta").mode("overwrite").save(f"s3a://{BUCKET_NAME}/database/top_performers_delta")